In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g4AmBo8LwVMUc0bqW7rqKiplu9ZD9YHSigXK9XjVkgV12Jb1RAnj2s
Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/DeepFake Detector'

/content/drive/MyDrive/Colab Notebooks/DeepFake Detector


In [3]:
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow_addons.losses import SigmoidFocalCrossEntropy

import keras
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import *
from keras.applications import *
from keras import metrics
from keras.losses import BinaryCrossentropy
from keras import backend as K

In [4]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
EPOCHS = 100
DATA_GENERATOR_SEED = 420
LEARNING_RATE = 5e-4
BACTH_SIZE = 64
VALIDATION_SPLIT = 0.1

DF_TYPEZ = ['diff', 'avg', 'rnd', 'diff-bal', 'avg-bal', 'rnd-bal'][::-1]
LOSSEZ = [BinaryCrossentropy()]
PRE_TRAINED_MODELZ = ['Xception', 'ResNet50', 'VGG19', 'InceptionV3']
OPTIMIZERZ = ['SGD', 'RMSprop', 'Adam']

In [ ]:
for DF_TYPE in tqdm(DF_TYPEZ):
    for LOSS in tqdm(LOSSEZ):
        for MDL in tqdm(PRE_TRAINED_MODELZ):
            for OPTIMIZER in tqdm(OPTIMIZERZ):
                
                EXPERIMENT_NAME = f'{DF_TYPE}-{MDL}-{OPTIMIZER}'.replace('-', '_').lower()

                try:
                    os.mkdir(f'\n./Checkpoints/{EXPERIMENT_NAME}')
                except OSError:
                    try:
                        print(f'loading model from ./Checkpoints/{EXPERIMENT_NAME}')
                        tf.saved_model.load(f'./Checkpoints/{EXPERIMENT_NAME}')
                    except: 
                        pass

                TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}'
                if 'bal' in TRAIN_VAL_DIR:
                    TEST_DIR = TRAIN_VAL_DIR.replace('bal', 'test')
                else:
                    TEST_DIR = f'{TRAIN_VAL_DIR}-test'

                TRAIN_DATAGEN = ImageDataGenerator(rescale = 1./255.,
                                                rotation_range = 40,
                                                width_shift_range = 0.2,
                                                height_shift_range = 0.2,
                                                shear_range = 0.2,
                                                zoom_range = 0.2,
                                                horizontal_flip = True,
                                                validation_split = VALIDATION_SPLIT)

                VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255., 
                                                validation_split = VALIDATION_SPLIT)

                TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.)

                TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                                    batch_size = BACTH_SIZE,
                                                                    class_mode = 'binary', 
                                                                    target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                                    subset = 'training',
                                                                    seed = DATA_GENERATOR_SEED)

                VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                                    batch_size = BACTH_SIZE,
                                                                    class_mode = 'binary', 
                                                                    target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                                    subset = 'validation',
                                                                    seed = DATA_GENERATOR_SEED)

                TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
                                                                batch_size = BACTH_SIZE,
                                                                class_mode = 'binary', 
                                                                target_size = (IMG_HEIGHT, IMG_WIDTH),                                
                                                                seed = DATA_GENERATOR_SEED)

                train_gen_list = list(TRAIN_GENERATOR.classes)
                val_gen_list = list(VALIDATION_GENERATOR.classes)
                test_gen_list = list(TEST_GENERATOR.classes)

                train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
                val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)
                test_neg, test_pos = test_gen_list.count(0), test_gen_list.count(1)

                pos = train_pos + val_pos + test_pos
                neg = train_neg + val_neg + test_neg
                total = pos + neg

                weight_for_0 = (1 / neg)*(total)/2.0 
                weight_for_1 = (1 / pos)*(total)/2.0

                CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}

                STEP_SIZE_TRAIN = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size
                STEP_SIZE_VALID = VALIDATION_GENERATOR.n//VALIDATION_GENERATOR.batch_size

                EARLY_STOPPING = EarlyStopping(monitor = 'auc', 
                                            patience = EPOCHS//2,
                                            mode = 'max',
                                            verbose = 1,
                                            restore_best_weights = True)
                
                REDUCE_LR  = ReduceLROnPlateau(monitor = 'auc', 
                                            patience = EPOCHS//6,
                                            factor = 0.5,
                                            verbose = 1,
                                            min_lr=1e-6)
                
                MODEL_CHECKPOINT = ModelCheckpoint(filepath = f'./Checkpoints/{EXPERIMENT_NAME}/',
                                                monitor = 'val_auc',
                                                mode = 'max',
                                                verbose = 1,
                                                save_best_only = True)
                
                TENSOR_BOARD = TensorBoard(log_dir = f'./Checkpoints/{EXPERIMENT_NAME}/logs')

                PRE_TRAINED_MODEL = eval(MDL)(input_shape = (IMG_HEIGHT,IMG_WIDTH,3),
                                                            include_top = False,
                                                            weights = 'imagenet')

                for layer in PRE_TRAINED_MODEL.layers:
                    layer.trainable = False

                x = layers.Flatten()(PRE_TRAINED_MODEL.output)  
                x = layers.Dense(1024, activation = 'relu')(x)
                x = layers.Dropout(0.2)(x)                  
                x = layers.Dense(1, activation = 'sigmoid')(x)  

                MODEL = Model(PRE_TRAINED_MODEL.input, x) 

                MODEL.compile(optimizer = eval(OPTIMIZER)(learning_rate = LEARNING_RATE),
                            loss = LOSS,
                            metrics = [metrics.BinaryAccuracy(name = 'acc'),
                                    metrics.AUC(name = 'auc'),
                                    metrics.FalsePositives(name = 'fp')])

                HISTORY = MODEL.fit(TRAIN_GENERATOR,
                                    epochs = EPOCHS,
                                    steps_per_epoch = STEP_SIZE_TRAIN,
                                    validation_data = VALIDATION_GENERATOR,
                                    validation_steps = STEP_SIZE_VALID,
                                    verbose = 1,
                                    class_weight = CLASS_WEIGHT,
                                    callbacks = [EARLY_STOPPING,
                                                REDUCE_LR,
                                                MODEL_CHECKPOINT,
                                                TENSOR_BOARD])

                acc = HISTORY.history['acc']
                auc = HISTORY.history['auc']
                loss = HISTORY.history['loss']
                fp = HISTORY.history['fp']

                val_acc = HISTORY.history['val_acc']
                val_auc = HISTORY.history['val_auc']
                val_loss = HISTORY.history['val_loss']
                val_fp = HISTORY.history['val_fp']

                epochs = range(len(acc))

                fig, axs = plt.subplots(2, 2, figsize = (20,20))

                axs[0, 0].plot(epochs, acc, 'r', label='Training Binary Accuracy')
                axs[0, 0].plot(epochs, val_acc, 'b', label='Validation Binary Accuracy')
                axs[0, 0].set_title('Training and Validation Binary Accuracy')
                axs[0, 0].legend()

                axs[0, 1].plot(epochs, loss, 'r', label='Training Loss')
                axs[0, 1].plot(epochs, val_loss, 'b', label='Validation Loss')
                axs[0, 1].set_title('Training and Validation Loss')
                axs[0, 1].legend()

                axs[1, 0].plot(epochs, auc, 'r', label='Training AUC')
                axs[1, 0].plot(epochs, val_auc, 'b', label='Validation AUC')
                axs[1, 0].set_title('Training and Validation AUROC')
                axs[1, 0].legend()

                axs[1, 1].plot(epochs, fp, 'r', label='Training False Positives')
                axs[1, 1].plot(epochs, val_fp, 'b', label='Validation False Positives')
                axs[1, 1].set_title('Training and Validation False Positives')
                axs[1, 1].legend()

                fig.savefig(f'./Checkpoints/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_plt.png')

                Y_pred = MODEL.predict(TEST_GENERATOR)
                Y_true = TEST_GENERATOR.classes

                text_file = open(f'./Checkpoints/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_summary.txt', 'w')

                text_file.write(f'\n{EXPERIMENT_NAME}\n')
                text_file.write(f'\nTRAIN\nReal samples = {train_neg}\nFake samples = {train_pos}')
                text_file.write(f'\nVALIDATION\nReal samples = {val_neg}\nFake samples = {val_pos}')
                text_file.write(f'\nTEST\nReal samples = {test_neg}\nFake samples = {test_pos}')
                text_file.write(f'\nWeight for Real = {weight_for_0:.5f}')
                text_file.write(f'\nWeight for Fake = {weight_for_1:.5f}')
                text_file.write(f'\nTrain step size = {STEP_SIZE_TRAIN}')
                text_file.write(f'\nValidation step size = {STEP_SIZE_VALID}')
                text_file.write(f'\nEarly Stop after {EARLY_STOPPING.patience} epochs of patience based on {EARLY_STOPPING.monitor}')
                text_file.write(f'\nModel Checkpoints saved to {MODEL_CHECKPOINT.filepath} based on {MODEL_CHECKPOINT.monitor}')
                text_file.write(f'\nLearning-Rate will reduce by a factor of {REDUCE_LR.factor} on {REDUCE_LR.monitor} plateu after {REDUCE_LR.patience} epochs of patience\n')
                text_file.write(f'\nTensor Board logs saved to {TENSOR_BOARD.log_dir}')

                div = 100
                for thrshld in map(lambda x: x/div, range(0,div+1)):
                    y_pred = (Y_pred > thrshld).astype(int)
                    text_file.write(f'\nTHRESHOLD = {thrshld}')
                    text_file.write(f'\nCONFUSION MATRIX\n{confusion_matrix(Y_true, y_pred)}')
                    text_file.write(f'\nCLASSIFICATION REPORT\n{classification_report(Y_true, y_pred, target_names = ["REAL", "FAKE"])}\n\n')
                    text_file.write('________________________________________________________________\n\n')
                    
                text_file.write(str(HISTORY.history).replace(', \'', ':\n\n'))
                text_file.close()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/3 [00:00<?, ?it/s]

loading model from ./Checkpoints/rnd_bal_xception_sgd
Found 760 images belonging to 2 classes.
Found 84 images belonging to 2 classes.
Found 1652 images belonging to 2 classes.
83689472/83683744 [==============================] - 1s 0us/step
Epoch 1/100
11/11 [==============================] - 306s 27s/step - loss: 0.8712 - acc: 0.4986 - auc: 0.4847 - fp: 27.1667 - val_loss: 0.8738 - val_acc: 0.5000 - val_auc: 0.5107 - val_fp: 0.0000e+00

Epoch 00001: val_auc improved from -inf to 0.51074, saving model to ./Checkpoints/rnd_bal_xception_sgd/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_xception_sgd/assets
Epoch 2/100
11/11 [==============================] - 11s 1s/step - loss: 0.7854 - acc: 0.4816 - auc: 0.5134 - fp: 2.1667 - val_loss: 0.9054 - val_acc: 0.5000 - val_auc: 0.3540 - val_fp: 0.0000e+00

Epoch 00002: val_auc did not improve from 0.51074
Epoch 3/100
11/11 [==============================] - 11s 1s/step - loss: 0.7745 - acc: 0.4898 - auc: 0.5350 - fp: 0.0000e+00 - v

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 33%|███▎      | 1/3 [36:37<1:13:15, 2197.67s/it]

loading model from ./Checkpoints/rnd_bal_xception_rmsprop
Found 760 images belonging to 2 classes.
Found 84 images belonging to 2 classes.
Found 1652 images belonging to 2 classes.
Epoch 1/100
11/11 [==============================] - 17s 1s/step - loss: 19.7526 - acc: 0.5224 - auc: 0.5234 - fp: 54.9167 - val_loss: 5.8555 - val_acc: 0.5000 - val_auc: 0.5469 - val_fp: 0.0000e+00

Epoch 00001: val_auc improved from -inf to 0.54688, saving model to ./Checkpoints/rnd_bal_xception_rmsprop/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_xception_rmsprop/assets
Epoch 2/100
11/11 [==============================] - 12s 1s/step - loss: 3.3857 - acc: 0.5143 - auc: 0.5070 - fp: 47.0000 - val_loss: 1.1429 - val_acc: 0.6094 - val_auc: 0.5992 - val_fp: 5.0000

Epoch 00002: val_auc improved from 0.54688 to 0.59922, saving model to ./Checkpoints/rnd_bal_xception_rmsprop/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_xception_rmsprop/assets
Epoch 3/100
11/11 [========================

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 67%|██████▋   | 2/3 [58:09<32:05, 1925.95s/it]  

...

loading model from ./Checkpoints/rnd_bal_vgg19_adam
Found 760 images belonging to 2 classes.
Found 84 images belonging to 2 classes.
Found 1652 images belonging to 2 classes.
Epoch 1/100
11/11 [==============================] - 13s 1s/step - loss: 4.4803 - acc: 0.4742 - auc: 0.4834 - fp: 50.5833 - val_loss: 2.7047 - val_acc: 0.5000 - val_auc: 0.5234 - val_fp: 0.0000e+00

Epoch 00001: val_auc improved from -inf to 0.52344, saving model to ./Checkpoints/rnd_bal_vgg19_adam/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_vgg19_adam/assets
Epoch 2/100
11/11 [==============================] - 11s 981ms/step - loss: 1.1369 - acc: 0.5677 - auc: 0.5713 - fp: 39.6667 - val_loss: 1.0851 - val_acc: 0.5156 - val_auc: 0.5474 - val_fp: 0.0000e+00

Epoch 00002: val_auc improved from 0.52344 to 0.54741, saving model to ./Checkpoints/rnd_bal_vgg19_adam/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_vgg19_adam/assets
Epoch 3/100
11/11 [==============================] - 11s 983ms/step

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



100%|██████████| 3/3 [57:47<00:00, 1155.67s/it]


 75%|███████▌  | 3/4 [3:28:54<1:11:46, 4306.53s/it]


  0%|          | 0/3 [00:00<?, ?it/s]

loading model from ./Checkpoints/rnd_bal_inceptionv3_sgd
Found 760 images belonging to 2 classes.
Found 84 images belonging to 2 classes.
Found 1652 images belonging to 2 classes.
87916544/87910968 [==============================] - 1s 0us/step
Epoch 1/100
11/11 [==============================] - 18s 1s/step - loss: 0.8959 - acc: 0.5170 - auc: 0.5748 - fp: 35.8333 - val_loss: 0.8412 - val_acc: 0.4844 - val_auc: 0.5020 - val_fp: 5.0000

Epoch 00001: val_auc improved from -inf to 0.50195, saving model to ./Checkpoints/rnd_bal_inceptionv3_sgd/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_inceptionv3_sgd/assets
Epoch 2/100
11/11 [==============================] - 12s 1s/step - loss: 0.8653 - acc: 0.5273 - auc: 0.5153 - fp: 17.5000 - val_loss: 0.9519 - val_acc: 0.4688 - val_auc: 0.5235 - val_fp: 2.0000

Epoch 00002: val_auc improved from 0.50195 to 0.52346, saving model to ./Checkpoints/rnd_bal_inceptionv3_sgd/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_inceptionv3

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 33%|███▎      | 1/3 [22:30<45:01, 1350.59s/it]

loading model from ./Checkpoints/rnd_bal_inceptionv3_rmsprop
Found 760 images belonging to 2 classes.
Found 84 images belonging to 2 classes.
Found 1652 images belonging to 2 classes.
Epoch 1/100
11/11 [==============================] - 19s 1s/step - loss: 43.6045 - acc: 0.4664 - auc: 0.4703 - fp: 71.4167 - val_loss: 15.2851 - val_acc: 0.5000 - val_auc: 0.5000 - val_fp: 0.0000e+00

Epoch 00001: val_auc improved from -inf to 0.50000, saving model to ./Checkpoints/rnd_bal_inceptionv3_rmsprop/
INFO:tensorflow:Assets written to: ./Checkpoints/rnd_bal_inceptionv3_rmsprop/assets
Epoch 2/100
11/11 [==============================] - 10s 929ms/step - loss: 8.2482 - acc: 0.4797 - auc: 0.5150 - fp: 48.9167 - val_loss: 26.0912 - val_acc: 0.4844 - val_auc: 0.5000 - val_fp: 0.0000e+00

Epoch 00002: val_auc did not improve from 0.50000
Epoch 3/100
11/11 [==============================] - 10s 930ms/step - loss: 10.8794 - acc: 0.4814 - auc: 0.4933 - fp: 34.3333 - val_loss: 16.4061 - val_acc: 0.5000 - v

In [ ]:
# IMG_HEIGHT, IMG_WIDTH = 224, 224
# EPOCHS = 4
# DATA_GENERATOR_SEED = 420
# LEARNING_RATE = 1e-4
# BACTH_SIZE = 32
# VALIDATION_SPLIT = 0.1
# LOSS = BinaryCrossentropy()
# DF_TYPE = 'diff'
# EXPERIMENT_NAME = f'{DF_TYPE}-{LOSS.name}'.replace('-', '_')

# TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}'
# if 'bal' in TRAIN_VAL_DIR:
#     TEST_DIR = TRAIN_VAL_DIR.replace('bal', 'test')
# else:
#     TEST_DIR = f'{TRAIN_VAL_DIR}-test'

In [ ]:
# TRAIN_DATAGEN = ImageDataGenerator(rescale = 1./255.,
#                                    rotation_range = 40,
#                                    width_shift_range = 0.2,
#                                    height_shift_range = 0.2,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True,
#                                    validation_split = VALIDATION_SPLIT)

# VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255., 
#                                  validation_split = VALIDATION_SPLIT)

# TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.)

# TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                     batch_size = BACTH_SIZE,
#                                                     class_mode = 'binary', 
#                                                     target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                     subset = 'training',
#                                                     seed = DATA_GENERATOR_SEED)


# VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                        batch_size = BACTH_SIZE,
#                                                        class_mode = 'binary', 
#                                                        target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                        subset = 'validation',
#                                                        seed = DATA_GENERATOR_SEED)

# TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
#                                                  batch_size = BACTH_SIZE,
#                                                  class_mode = 'binary', 
#                                                  target_size = (IMG_HEIGHT, IMG_WIDTH),                                
#                                                  seed = DATA_GENERATOR_SEED)

In [ ]:
# train_gen_list = list(TRAIN_GENERATOR.classes)
# val_gen_list = list(VALIDATION_GENERATOR.classes)
# test_gen_list = list(TEST_GENERATOR.classes)

# train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
# val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)
# test_neg, test_pos = test_gen_list.count(0), test_gen_list.count(1)

# pos = train_pos + val_pos + test_pos
# neg = train_neg + val_neg + test_neg
# total = pos + neg

# weight_for_0 = (1 / neg)*(total)/2.0 
# weight_for_1 = (1 / pos)*(total)/2.0

# CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}

# STEP_SIZE_TRAIN = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size
# STEP_SIZE_VALID = VALIDATION_GENERATOR.n//VALIDATION_GENERATOR.batch_size

# EARLY_STOPPING = EarlyStopping(monitor = 'fp', 
#                                patience = EPOCHS//2,
#                                mode = 'max',
#                                restore_best_weights = True)

# MODEL_CHECKPOINT = ModelCheckpoint(filepath = f'./Checkpoints/{EXPERIMENT_NAME}/',
#                                    monitor='val_loss',
#                                    save_best_only=True)

# print(f'\nTRAIN\nReal samples = {train_neg}\nFake samples = {train_pos}')
# print(f'\nVALIDATION\nReal samples = {val_neg}\nFake samples = {val_pos}')
# print(f'\nTEST\nReal samples = {test_neg}\nFake samples = {test_pos}')
# print(f'\nWeight for Real = {weight_for_0:.5f}')
# print(f'Weight for Fake = {weight_for_1:.5f}')
# print(f'\nTrain step size = {STEP_SIZE_TRAIN}')
# print(f'Validation step size = {STEP_SIZE_VALID}')
# print(f'\nEarly Stop after {EARLY_STOPPING.patience} epochs of patience based on {EARLY_STOPPING.monitor}')
# print(f'Model Checkpoints saved to {MODEL_CHECKPOINT.filepath} based on {MODEL_CHECKPOINT.monitor}')

In [ ]:
# PRE_TRAINED_MODEL = InceptionV3(input_shape = (IMG_HEIGHT,IMG_WIDTH,3),
#                                 include_top = False,
#                                 weights = 'imagenet')

# for layer in PRE_TRAINED_MODEL.layers:
#     layer.trainable = False

# x = layers.Flatten()(PRE_TRAINED_MODEL.output)  
# x = layers.Dense(1024, activation = 'relu')(x)
# x = layers.Dropout(0.2)(x)                  
# x = layers.Dense(1, activation = 'sigmoid')(x)  

# MODEL = Model(PRE_TRAINED_MODEL.input, x) 

# MODEL.compile(optimizer = Adam(lr = LEARNING_RATE),
#             loss = LOSS,
#             metrics = [metrics.BinaryAccuracy(name = 'acc'),
#                        metrics.AUC(name = 'auc'),
#                        metrics.FalsePositives(name = 'fp')])

In [ ]:
# HISTORY = MODEL.fit(TRAIN_GENERATOR,
#                     epochs = EPOCHS,
#                     steps_per_epoch = STEP_SIZE_TRAIN,
#                     validation_data = VALIDATION_GENERATOR,
#                     validation_steps = STEP_SIZE_VALID,
#                     verbose = 2,
#                     class_weight = CLASS_WEIGHT,
#                     callbacks = [EARLY_STOPPING,
#                                  MODEL_CHECKPOINT])

In [ ]:
# history = {'loss': [1.208497405052185, 1.267728567123413], 'acc': [0.5041208863258362, 0.49862638115882874], 'auc': [0.500086784362793, 0.5], 'fp': [6.0, 0.0], 'val_loss': [1.9281185865402222, 1.7473573684692383], 'val_acc': [0.5, 0.546875], 'val_auc': [0.5, 0.5], 'val_fp': [0.0, 0.0]}
# acc = history['acc']
# auc = history['auc']
# loss = history['loss']
# fp = history['fp']

# val_acc = history['val_acc']
# val_auc = history['auc']
# val_loss = history['val_loss']
# val_fp = history['val_fp']

In [ ]:
# acc = HISTORY.history['acc']
# auc = HISTORY.history['auc']
# loss = HISTORY.history['loss']
# fp = HISTORY.history['fp']

# val_acc = HISTORY.history['val_acc']
# val_auc = HISTORY.history['auc']
# val_loss = HISTORY.history['val_loss']
# val_fp = HISTORY.history['val_fp']

# epochs = range(len(acc))

# fig, axs = plt.subplots(2, 2, figsize=(15,15))

# axs[0, 0].plot(epochs, acc, 'r', label='Training Binary Accuracy')
# axs[0, 0].plot(epochs, val_acc, 'b', label='Validation Binary Accuracy')
# axs[0, 0].set_title('Training and Validation Binary Accuracy')
# axs[0, 0].legend()

# axs[0, 1].plot(epochs, loss, 'r', label='Training Loss')
# axs[0, 1].plot(epochs, val_loss, 'b', label='Validation Loss')
# axs[0, 1].set_title('Training and Validation Loss')
# axs[0, 1].legend()

# axs[1, 0].plot(epochs, auc, 'r', label='Training AUC')
# axs[1, 0].plot(epochs, val_auc, 'b', label='Validation AUC')
# axs[1, 0].set_title('Training and Validation AUROC')
# axs[1, 0].legend()

# axs[1, 1].plot(epochs, fp, 'r', label='Training False Positives')
# axs[1, 1].plot(epochs, val_fp, 'b', label='Validation False Positives')
# axs[1, 1].set_title('Training and Validation False Positives')
# axs[1, 1].legend()

# fig.savefig(f'./Checkpoints/{EXPERIMENT_NAME}/plt.png')

In [ ]:
# Y_pred_test = MODEL.predict(TEST_GENERATOR) 

In [ ]:
# Y_pred_val = MODEL.predict(VALIDATION_GENERATOR) 

In [ ]:
# Y_pred = np.concatenate((Y_pred_test, Y_pred_val), axis=0)
# Y_true = np.concatenate((TEST_GENERATOR.classes, VALIDATION_GENERATOR.classes), axis=0)

In [ ]:
# div = 20
# for thrshld in map(lambda x: x/div, range(0,div+1)):
#     y_pred = (Y_pred > thrshld).astype(int)
#     print(f'THRESHOLD = {thrshld}')
#     print(f'\nCONFUSION MATRIX\n{confusion_matrix(Y_true, y_pred)}')
#     print(f'\nCLASSIFICATION REPORT\n{classification_report(Y_true, y_pred, target_names = ["REAL", "FAKE"])}\n\n')
#     print('________________________________________________________________')

In [ ]:
# text_file = open(f'./Checkpoints/{EXPERIMENT_NAME}/summary.txt', 'w')
# text_file.write(str(HISTORY.history).replace(', \'', ':\n\n'))
# text_file.close()